This example uses the default GLiNER model to perform named entity recognition.



In [19]:
from pydantic import BaseModel, field_validator
from typing import List

class Entity(BaseModel):
    entity: str
    types: str
    
    @field_validator('entity')
    @classmethod
    def clean_entity(cls, v):
        # Take only the text before the comma if there is one
        if isinstance(v, str) and "," in v:
            return v.split(",")[0].strip()
        return v

class Result(BaseModel):
    text: str
    entities: List[Entity]

In [20]:
import json
from pathlib import Path

test_set_path = Path("results.jsonl")

results = []

with open(test_set_path, "r") as f:
    for line in f:
        raw_data = json.loads(line)
        try:
            result = Result(**raw_data)
            results.append(result)
        except Exception as e:
            print(f"Error parsing line: {e}")
            print(f"Problematic data: {raw_data}")

In [ ]:
from gliner import GLiNER

model = GLiNER.from_pretrained("urchade/gliner_large-v2.1", local_files_only= True)

labels = ["Interval", "Money", "Start Date", "Phone", "Address", "Person", "Faculty"]

for result in results:
    text = result.text
    print(text)
    # Use the correct API for text input - GLiNER expects text directly
    entities = model.predict_entities(text, labels, threshold=0.8)

    # Display predicted entities and their labels
    for entity in entities:
        print(entity["text"], "=>", entity["label"])


C:\Users\maish\AppData\Roaming\Python\Python312\site-packages\transformers\convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Dear University of Manitoba Development Office,
I am delighted to be supporting your cause through a one-time gift of $20,000.
I would like this donation to go towards the Computer Science Department's Fellowship Program. Please let me know if this is feasible and how we can make it happen. 
Thank you for your time and consideration.
Sincerely,
Michael Roberts.


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


$20,000 => Money
Computer Science Department => Faculty
Sincerely => Address
Michael Roberts => Person
Dear Friends at the University of Manitoba Foundation,

I am writing to express my interest in establishing a bursary for students studying Music at the university. I would like to make a one-time gift of $15,000 and a recurring gift of $1,000 every two years.

Please let me know if this is feasible and what process I need to follow to set up these gifts.

You can contact me at 204-456-1234 or via email at donorsmith@donorsmith.org.

Thank you for your time and consideration.
$15,000 => Money
$1,000 => Money
every two years => Interval
204-456-1234 => Phone
donorsmith@donorsmith.org => Address
Dear University of Manitoba Development Office, I would like to set up a one-time gift to support the Civil Engineering Bursary. The amount will be $5,000 and it will be transferred via wire transfer. Could you please provide me with the necessary information on how to proceed? My contact inform

KeyboardInterrupt: 